In [ ]:
# Fast Food Segmentation Analysis using Python

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data
df = pd.read_csv('mcdonalds.csv')

# Step 1: Convert Yes/No columns to binary (first 11 columns)
binary_cols = df.columns[:11]
df[binary_cols] = df[binary_cols].applymap(lambda x: 1 if x == 'Yes' else 0)

# Step 2: Convert 'Like' column to numerical values
like_scale = {
    "I hate it!-5": -5, "-4": -4, "-3": -3, "-2": -2, "-1": -1,
    "0": 0, "+1": 1, "+2": 2, "+3": 3, "+4": 4, "I love it!+5": 5
}
if 'Like' in df.columns:
    df['Like.n'] = df['Like'].map(like_scale)

# Step 3: Perform PCA on binary features
pca = PCA(n_components=2)
X_pca = pca.fit_transform(df[binary_cols])

# Step 4: Plot perceptual map
plt.figure(figsize=(8,6))
plt.scatter(X_pca[:, 0], X_pca[:, 1], alpha=0.3, color='darkorange')
plt.title('Perceptual Map (PCA)')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.grid(True)
plt.tight_layout()
plt.show()

# Step 5: Apply K-Means Clustering
kmeans = KMeans(n_clusters=4, random_state=42)
df['Cluster'] = kmeans.fit_predict(df[binary_cols])

# Step 6: Profile Plot of Segments
profile_means = df.groupby('Cluster')[binary_cols].mean().T

plt.figure(figsize=(10,6))
for i in range(profile_means.shape[1]):
    plt.plot(profile_means.index, profile_means.iloc[:, i], marker='o', label=f'Segment {i}')
plt.xticks(rotation=45)
plt.ylabel('Proportion Saying "Yes"')
plt.title('Cluster Profile Plot Based on Perceptions')
plt.legend()
plt.tight_layout()
plt.show()
